In [ ]:
import string
from pathlib import Path

import cairosvg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skunk
import yaml
from coulson.draw import draw_mol
from matplotlib.offsetbox import AnnotationBbox
from rdkit import Chem
from utils import (
    AXIS_LABELS,
    PARENT_SMILES,
    add_to_variables,
    crop_image,
    format_dictionary_for_yaml,
    load_data,
)

plt.style.use("default")

Set data to save

In [ ]:
variables = {}
params = {}

Load data

In [ ]:
path_azaphenalenes = snakemake.input.azaphenalenes
path_azulenes = snakemake.input.azulenes
path_rational_design = snakemake.input.rational_design

path_azaphenalenes_ref = snakemake.input.azaphenalenes_ref
path_azulenes_ref = snakemake.input.azulenes_ref
path_rational_design_ref = snakemake.input.rational_design_ref

df_azaphenalenes = load_data(path_azaphenalenes, path_azaphenalenes_ref).dropna()
df_azulenes = load_data(path_azulenes, path_azulenes_ref).dropna()
df_rational_design = load_data(path_rational_design, path_rational_design_ref).dropna()

Plotting settings

In [ ]:
x_name = "t1_s1_ref"
y_name = "t1_s1_dsp_cis"
x_label = AXIS_LABELS[x_name]
y_label = AXIS_LABELS[y_name]
label_positive = r"Positive $\Delta E_T$"
label_negative = r"Negative $\Delta E_T$"
plot_kwargs = {"edgecolors": "black", "linewidths": 0.5}

## Analyze number of negative excitation energies

In [ ]:
jobs = (
    ("Azaphenalenes", df_azaphenalenes),
    ("Azulenes", df_azulenes),
    ("Rational design", df_rational_design),
)

fig, ax = plt.subplots(1, 3, figsize=plt.rcParams["figure.figsize"] * np.array([3, 1]))

# Create subfigure labels
for i, ax_ in enumerate(ax):
    label = string.ascii_lowercase[i]
    ax_.text(
        0.05,
        0.95,
        f"({label})",
        fontsize="large",
        transform=ax_.transAxes,
        verticalalignment="top",
    )

all_results = {}
for i, (
    name,
    df,
) in enumerate(jobs):
    n_neg_t1 = (df["t1_cis"] < 0).sum()
    n_neg_s1 = (df["s1_cis"] < 0).sum()
    n_total = df.shape[0]
    results = {
        "N(neg. triplet)": n_neg_t1,
        "N(neg. singlet)": n_neg_s1,
        "N(total)": n_total,
        "%(neg. triplet)": n_neg_t1 / n_total * 100,
        "%(neg. singlet)": n_neg_s1 / n_total * 100,
    }
    all_results[name] = results

    variables = add_to_variables(
        variables,
        results,
        label_calculation="negative",
        label_compound="all",
        label_method="_".join(name.lower().split()),
    )

    neg_mask = df["t1_cis"] < 0
    df[~neg_mask].plot.scatter(
        x="t1_s1_ref",
        y="t1_s1_dsp_cis",
        c="C0",
        ax=ax[i],
        **plot_kwargs,
    )
    df[neg_mask].plot.scatter(
        x="t1_s1_ref",
        y="t1_s1_dsp_cis",
        c="C1",
        ax=ax[i],
        **plot_kwargs,
    )
    ax[i].set_xlabel(x_label)
    ax[i].set_ylabel(y_label)
    ax[i].set_title(name)
fig.tight_layout()
fig.legend(
    [label_positive, label_negative],
    loc="lower center",
    ncol=2,
    bbox_to_anchor=(0.5, -0.07),
    bbox_transform=fig.transFigure,
)

# Save figure
path_figure = snakemake.output.figure_negative_all
variables[f"fig_negative_all"] = "../" + str(Path(path_figure).with_suffix(""))
fig.savefig(path_figure)

# Save data to table
path_table = snakemake.output.table_negative
params[f"tab_negative"] = "../" + path_table

# Print tables for the paper
df = pd.DataFrame(all_results).T
floatfmt = [None] + [".0f"] * 3 + [".2f"] * 2
df.replace(np.nan, None).to_markdown(path_table, floatfmt=floatfmt, missingval="-")

## Rational design

In [ ]:
# Take out scaffold names
df_rational_design["scaffold"] = [idx.split("_")[0] for idx in df_rational_design.index]
scaffolds = list(PARENT_SMILES.keys())

# Make plot
n_rows = 4
n_columns = 4
fig, axs = plt.subplots(
    n_columns,
    n_rows,
    figsize=plt.rcParams["figure.figsize"] * np.array([n_rows, n_columns]),
)

# Loop over scaffolds
scaffold_svgs = {}
for i, scaffold in enumerate(scaffolds):
    df = df_rational_design[df_rational_design["scaffold"] == scaffold]
    neg_mask = df["t1_cis"] < 0

    i_row = i // n_rows
    i_col = i % n_columns
    ax = axs[i_row, i_col]
    smiles = PARENT_SMILES[scaffold]
    mol = Chem.MolFromSmiles(smiles)
    mol_image = draw_mol(mol, img_format="svg")

    df[~neg_mask].plot.scatter(x=x_name, y=y_name, c="C0", ax=ax, **plot_kwargs)
    df[neg_mask].plot.scatter(x=x_name, y=y_name, c="C1", ax=ax, **plot_kwargs)
    ax.set_title(scaffold)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

    skunk_label = f"sk_{i}"

    box = skunk.Box(50, 50, gid=skunk_label)
    ab = AnnotationBbox(
        box, (1, 0), xycoords="axes fraction", box_alignment=(1.1, -0.1)
    )
    ax.add_artist(ab)
    scaffold_svgs[skunk_label] = mol_image
fig.tight_layout()
fig.legend(
    [label_positive, label_negative],
    loc="lower center",
    ncol=2,
    bbox_to_anchor=(0.5, -0.02),
    bbox_transform=fig.transFigure,
)

# Insert SVGs of molecules
svg = skunk.insert(scaffold_svgs)
png = cairosvg.svg2png(bytestring=svg, dpi=300, background_color="white")
png_cropped = crop_image(png)

path_figure = snakemake.output.figure_negative_scaffolds
with open(path_figure, "wb") as f:
    f.write(png_cropped)
variables[f"fig_negative_scaffolds"] = "../" + str(Path(path_figure).with_suffix(""))

Save the parameters and variables to file

In [ ]:
with open(snakemake.output.params, "w") as f:
    yaml.dump(format_dictionary_for_yaml(params, n_dec=2), f)
with open(snakemake.output.variables, "w") as f:
    yaml.dump(format_dictionary_for_yaml(variables, n_dec=2), f)